In [1]:
import logging
# logging.basicConfig(level=logging.DEBUG)
from openmxr.song import Song

Output()

In [2]:


song: Song = Song.find(spotify_link="https://open.spotify.com/track/6l9Vr4mIGOH2MHnxyiqUGY")

array([ 2.41287193e-03, -6.60464620e-03, -4.84900443e-03, ...,
       -2.97506517e-05,  2.98429683e-05, -1.95980130e-05])

In [ ]:
if not song.youtube_meta["heatmap"]:
    pass
items = sorted(song.youtube_meta["heatmap"], reverse=True, key=lambda x:x["value"])[:3]
heat_moments = [int(i["start_time"] * song.sample_rate+i["end_time"] * song.sample_rate)//2 for i in items]

In [ ]:
heat_moments

In [ ]:
from scipy.signal import resample

y2 = resample(song.audio[0], (len(song.audio[0])//song.sample_rate)*44100)

In [ ]:

from madmom.features.downbeats import DBNDownBeatTrackingProcessor, RNNDownBeatProcessor

downbeats_proc = DBNDownBeatTrackingProcessor(beats_per_bar=[4], fps=100)
activations = RNNDownBeatProcessor()

activations = activations(y2, sample_rate=44100, num_channels=1)
downbeats = downbeats_proc(activations)

downbeats

In [ ]:
import librosa
beats = librosa.beat.beat_track(y=song.audio[:, 0], sr=song.sample_rate)
bpm = beats[0]

bpm

In [ ]:
import numpy as np


chromagram = librosa.feature.chroma_stft(y=song.audio[:, 0], sr=song.sample_rate)
mean_chroma = np.mean(chromagram, axis=1)
chroma_to_key = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
estimated_key_index = np.argmax(mean_chroma)
estimated_key = chroma_to_key[estimated_key_index]
estimated_key


In [ ]:
mapped = [min([int(i*song.sample_rate) for i in downbeats[:, 0]], key=lambda x:abs(x-hotspot)) for hotspot in heat_moments]
[a/song.sample_rate for a in mapped]